In [726]:
from tabulate import tabulate
import numpy as np

import random

n_periods: int = 8

team_members: list[str] = [
    "Benjamin",
    "Vik",
    "Vigo",
    "Can",
    "Njabulo",
    "Lars",
    "Sebas",
    "Kees",
    "Hamza",
]

excl_team: list[str] = []

tasks_base = [
    "Keeper",
    "Aanvaller",
    "Middenvelder 1",
    "Middenvelder 2",
    "Verdediger 1",
    "Verdediger 2",
]

headers = [f"Periode {i + 1}" for i in range(n_periods)]

keepers: list[str] = ["Hamza", "Vik", "Vigo", "Njabulo"]
keeper_idx = [team_members.index(keeper) for keeper in keepers]
random.shuffle(keeper_idx)

team: list[str] = [i for i in team_members if i not in excl_team]
team_idx = [team_members.index(member) for member in team]
random.shuffle(team_idx)

n_team: int = len(team)  # Number of team members
n_tasks: int = len(tasks_base)  # Number of team positions
n_subs: int = n_team - n_tasks  # The number of subs

tasks_sub = [f"Wissel {i}" for i in range(1, n_subs + 1)]
tasks = tasks_base + tasks_sub

# Create the matrix where rows are n team members playing and columns the
# n of periods
matrix: np.array = np.full((n_team, n_periods), -1).astype(int)

In [727]:
# Set the keepers
matrix[0, np.arange(len(keeper_idx)) * 2] = keeper_idx
matrix[0, np.arange(len(keeper_idx)) * 2 + 1] = keeper_idx

matrix

array([[ 2,  2,  1,  1,  4,  4,  8,  8],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1]])

In [728]:
# Set the substitutes for each period
sub_list: list = team_idx * (n_subs + 1)

for i in range(n_periods):
    for j in range(1, n_subs + 1):
        # t represents the first team member id from the list    
        t: int = sub_list[0]

        # Check if the team member is already in the column
        if np.isin(t, matrix[:,i]):
            # Take the next element in the list given t is Keeper    
            t = sub_list[1]
            # Check if team member was sub in previous period, if yes take next
            if np.isin(t, matrix[-n_subs:,i-1]):
                t = sub_list.pop(2)
            else:
                t = sub_list.pop(1)
            matrix[j * -1, i] = t
        else:
            sub_list.remove(t)
            matrix[j * -1, i] = t        

matrix

array([[ 2,  2,  1,  1,  4,  4,  8,  8],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1],
       [ 4,  3,  7,  4,  1,  7,  2,  3],
       [ 8,  1,  5,  8,  0,  5,  4,  1],
       [ 6,  0,  2,  6,  2,  3,  6,  0]])

In [729]:
# Aanvallers for each period
for i in range(n_periods):
        
    unique_values, counts = np.unique(np.append(matrix[1,:], team_idx), 
                                      return_counts=True)

    # Sort the unique values and their counts
    sorted_indices = np.argsort(counts)
    sorted_values = unique_values[sorted_indices]
    sorted_counts = counts[sorted_indices]
    print(f"Loop number: {i}")
    # for value, count in zip(sorted_values, sorted_counts):
    #     print(f"Value: {value}, Count: {count}")

    val = np.array([el for el in sorted_values if el not in matrix[:,i]])

    matrix[1,i] = val[0]
    

matrix[1,:]


Loop number: 0
[ 0  1  2  3  4  5  6  7  8 -1]
[0 1 3 5 7]
Loop number: 1
[ 1  2  3  4  5  6  7  8  0 -1]
[4 5 6 7 8]
Loop number: 2
[ 1  2  3  5  6  7  8  0  4 -1]
[3 6 8 0 4]
Loop number: 3
[ 1  2  5  6  7  8  0  3  4 -1]
[2 5 7 0 3]
Loop number: 4
[ 1  5  6  7  8  0  2  3  4 -1]
[5 6 7 8 3]
Loop number: 5
[ 1  6  7  8  0  2  3  4  5 -1]
[1 6 8 0 2]
Loop number: 6
[ 6  7  8 -1  0  1  2  3  4  5]
[7 0 1 3 5]
Loop number: 7
[-1  6  8  0  1  2  3  4  5  7]
[6 2 4 5 7]


array([0, 4, 3, 2, 5, 1, 7, 6])

In [730]:
# Replace the index with the name of team members
matrix_str = [[team_members[matrix[i][j]] for j in range(n_periods)] for i in range(n_team)]

# Add tasks row
[row.insert(0, task) for task, row in zip(tasks, matrix_str)]

print(tabulate(matrix_str, headers=headers, tablefmt="grid"))

+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|                | Periode 1   | Periode 2   | Periode 3   | Periode 4   | Periode 5   | Periode 6   | Periode 7   | Periode 8   |
+================+=============+=============+=============+=============+=============+=============+=============+=============+
| Keeper         | Vigo        | Vigo        | Vik         | Vik         | Njabulo     | Njabulo     | Hamza       | Hamza       |
+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| Aanvaller      | Benjamin    | Njabulo     | Can         | Vigo        | Lars        | Vik         | Kees        | Sebas       |
+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| Middenvelder 1 | Hamza       | Hamza       | Hamza       | Hamza       | Hamza   

In [731]:

unique_values, counts = np.unique(matrix[-n_subs:, :], return_counts=True)

# Combine the unique values and their counts into a dictionary for an overview
for value, count in zip(unique_values, counts):
    print(f"Value {team_members[value]}: {count} occurrences")




Value Benjamin: 3 occurrences
Value Vik: 3 occurrences
Value Vigo: 3 occurrences
Value Can: 3 occurrences
Value Njabulo: 3 occurrences
Value Lars: 2 occurrences
Value Sebas: 3 occurrences
Value Kees: 2 occurrences
Value Hamza: 2 occurrences


In [732]:
# Replace the index with the name of team members
matrix_str = [[team_members[matrix[i][j]] for j in range(n_periods)] for i in range(n_team)]

# Add tasks row
[row.insert(0, task) for task, row in zip(tasks, matrix_str)]

print(tabulate(matrix_str, headers=headers, tablefmt="grid"))

+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
|                | Periode 1   | Periode 2   | Periode 3   | Periode 4   | Periode 5   | Periode 6   | Periode 7   | Periode 8   |
+================+=============+=============+=============+=============+=============+=============+=============+=============+
| Keeper         | Vigo        | Vigo        | Vik         | Vik         | Njabulo     | Njabulo     | Hamza       | Hamza       |
+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| Aanvaller      | Benjamin    | Njabulo     | Can         | Vigo        | Lars        | Vik         | Kees        | Sebas       |
+----------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+-------------+
| Middenvelder 1 | Hamza       | Hamza       | Hamza       | Hamza       | Hamza   